In [ ]:
from IPython.display import clear_output 
!pip install ghostscript
!pip install camelot-py[cv]
!pip install excalibur-py
!apt install ghostscript python3-tk
clear_output()

In [ ]:
import re


def extractTDTP(val):
  txts = re.split("G[0-9]:",val)
  ret = []
  for i in range(1,len(txts)):
    ext = txts[i].replace("\n"," ")
    NomModule = ext.split(") ")[1].split(",")[0].replace("\n"," ")
    SalleModule = ext.split(") ")[0].split("(")[1]
    NomProfModule = ext.split(",")[-1].replace("\n"," ")
    grp = re.findall("G[0-9]",val)

    toAdd = "TD"
    if "TP" in val:
      toAdd = "TP"

    ret.append([NomModule,SalleModule,NomProfModule,grp[i-1],toAdd])

  return ret

def extractCour(val):
  v = re.split("cours.+\(.+\)",val)

  NomModule = v[0].replace("\n"," ")
  SalleModule = re.search("\(.+\)",val).group(0)
  NomProfModule = v[1].replace("\n"," ")
  
  return [[NomModule,SalleModule,NomProfModule]]
def detect(val):
  if ("cours" in val):
    return 0
  else :
    if (val == ""):
      return 2
    else:
      return 1

def getEDT (data):
  edt = []
  for i in range(1,len(data)):
    for j in range(1,len(data[i])):
      rType = detect(data[i][j])
      if(rType == 0):
        edt.append([data[i][0],data[0][j].replace('\n',' ') , extractCour(data[i][j])])
      else:
        if(rType == 1):
          edt.append([data[i][0],data[0][j].replace('\n',' ') , extractTDTP(data[i][j])])
        else:
          edt.append([data[i][0],data[0][j].replace('\n',' ') , [[]]])
  return edt

def formating(s):
  res = ""
  b = False
  i = 0
  while i < len(s):
    if s[i] == " ":
      if b :
        res = res + " "
        b = False
      i = i + 1
    else:
      b = True
      res = res + s[i]
      i = i + 1

  if res == "":
    return ""
  if(res[-1] == " "):
    return res[0:-1]
  return res

def cleanEDT(edt):
  for i in range(len(edt)):
    for j in range(len(edt[i][2])):
      for k in range(len(edt[i][2][j])):
        edt[i][2][j][k] = formating(edt[i][2][j][k])
    return edt

In [ ]:
import json

def getJson(F,G,S,edt):
  Dict = {
    'Filere': F,
    'Grade': G,
    'Section' : S,
    'Jour' : {
        "Sam" : {},
        "Dim" : {},
        "Lun" : {},
        "Mar" : {},
        "Mer" : {},
        "Jeu" : {}
    }
  }

  for i in range(len(edt)):
    l1 = []
    for j in range(len(edt[i][2])):
      d = {}
      if edt[i][2][j] != []:
        d.update({"Module":""+edt[i][2][j][0]})
        d.update({"Salle":""+edt[i][2][j][1]})
        d.update({"Prof":""+edt[i][2][j][2]})
        if (len(edt[i][2][j]) == 5):
          d.update({"Groupe":""+edt[i][2][j][3]})
          d.update({"Type":""+edt[i][2][j][4]})
          
        l1.append(d)
    if l1 == []:
      l1 = None
    Dict["Jour"][edt[i][0]].update({""+edt[i][1]: l1})

  return Dict

def correct(dt):
  days = ["Sam","Dim","Lun","Mar","Mer","Jeu"]
  for i in range(1,7):
    dt.df[0][i] = days[i-1]

  for i in range(1,7):
    for d in days:
      if d in dt.df[1][i]:
        dt.df[1][i] = dt.df[1][i].replace(d,"")

In [ ]:
import camelot

all = []

In [ ]:
file = "/content/edt/L2-L3-ACAD-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])
correct(tables[2])
correct(tables[3])
correct(tables[4])
correct(tables[5])

all.append(getJson("ACAD","L2","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("ACAD","L2","B",cleanEDT(getEDT(tables[1].df.T))))
all.append(getJson("ACAD","L2","C",cleanEDT(getEDT(tables[2].df.T))))
all.append(getJson("ACAD","L3","A",cleanEDT(getEDT(tables[3].df.T))))
all.append(getJson("ACAD","L3","B",cleanEDT(getEDT(tables[4].df.T))))
all.append(getJson("ACAD","L3","C",cleanEDT(getEDT(tables[5].df.T))))

print(len(all))

6


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-7
  warnings.warn(


In [ ]:
file = "/content/edt/L2-L3-GTR-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])

all.append(getJson("GTR","L3","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("GTR","L2","A",cleanEDT(getEDT(tables[1].df.T))))
print(len(all))

8


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-3
  warnings.warn(


In [ ]:
file = "/content/edt/L2-L3-ISIL-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])
correct(tables[2])
correct(tables[3])

all.append(getJson("ISIL","L2","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("ISIL","L2","B",cleanEDT(getEDT(tables[1].df.T))))
all.append(getJson("ISIL","L3","A",cleanEDT(getEDT(tables[2].df.T))))
all.append(getJson("ISIL","L3","B",cleanEDT(getEDT(tables[3].df.T))))
print(len(all))

12


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-5
  warnings.warn(


In [ ]:
file = "/content/edt/M1-HPC-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])

all.append(getJson("HPC","M1","A",cleanEDT(getEDT(tables[0].df.T))))
print(len(all))

13


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-2
  warnings.warn(


In [ ]:
file = "/content/edt/M1-M2-BIGDATA-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])

all.append(getJson("BIGDATA","M1","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("BIGDATA","M2","A",cleanEDT(getEDT(tables[1].df.T))))

print(len(all))

15


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-3
  warnings.warn(


In [ ]:
file = "/content/edt/M1-M2-BIOINFO-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])

all.append(getJson("BIOINFO","M1","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("BIOINFO","M2","A",cleanEDT(getEDT(tables[1].df.T))))

print(len(all))

17


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-3
  warnings.warn(


In [ ]:
file = "/content/edt/M1-M2-IL-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])

all.append(getJson("IL","M2","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("IL","M1","A",cleanEDT(getEDT(tables[1].df.T))))

print(len(all))

19


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-3
  warnings.warn(


In [ ]:
file = "/content/edt/M1-M2-MIV-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])

all.append(getJson("IV","M2","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("IV","M1","A",cleanEDT(getEDT(tables[1].df.T))))

print(len(all))

21


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-3
  warnings.warn(


In [ ]:
file = "/content/edt/M1-M2-RSD-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])

all.append(getJson("RSD","M1","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("RSD","M2","A",cleanEDT(getEDT(tables[1].df.T))))

print(len(all))

23


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-3
  warnings.warn(


In [ ]:
file = "/content/edt/M1-M2-SII-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])

all.append(getJson("SII","M1","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("SII","M2","A",cleanEDT(getEDT(tables[1].df.T))))

print(len(all))

25


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-3
  warnings.warn(


In [ ]:
file = "/content/edt/M1-M2-SSI-1.pdf"
tables = camelot.read_pdf(file, pages='1-end')

correct(tables[0])
correct(tables[1])

all.append(getJson("SSI","M1","A",cleanEDT(getEDT(tables[0].df.T))))
all.append(getJson("SSI","M2","A",cleanEDT(getEDT(tables[1].df.T))))

print(len(all))

27


/usr/local/lib/python3.8/dist-packages/camelot/parsers/lattice.py:416: UserWarning: No tables found on page-3
  warnings.warn(


In [ ]:
with open("sample.json", "w") as outfile:
    json.dump(all, outfile)